HDDM analysis of pre- and post-treatment data from 43 HCs and 38 MDD adolescents

[Cleaning](#Cleaning)  
[Baseline](#Baseline)   
[Follow Up](#Follow-Up)  

In [3]:
import datetime, mmap,os,re, sys, decimal, hddm
import numpy as np 
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import scipy as sp
from patsy import dmatrix
%matplotlib inline
import warnings 
warnings.filterwarnings('ignore')
os.environ['R_HOME'] = '/Users/mlm2/anaconda/envs/pythonthreefive/lib/R' # couldn't find R in this environment for some reason
% matplotlib inline 
% load_ext rpy2.ipython
%R library(lsmeans)
%R library(lmerTest)
%R library(afex)
%R library(ppcor)
%R library(cocor)

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


array(['cocor', 'ppcor', 'MASS', 'afex', 'reshape2', 'lmerTest', 'lme4',
       'Matrix', 'lsmeans', 'estimability', 'tools', 'stats', 'graphics',
       'grDevices', 'utils', 'datasets', 'methods', 'base'], 
      dtype='<U12')

In [ ]:
path2data = '/Users/victorialawlor/Work/Expts/KIDPRT/Data/'

# Cleaning

# Baseline

### Participant Characteristics

### Standard Analysis

### HDDM

# Follow-up

### Participant Characteristics

### Standard Analysis

### HDDM

# HDDM Stability